**Цель задания**: научиться на простейшем уровне подбирать архитектуру полносвязной нейронной сети для получения качества решения задачи не ниже заданного.

**Задание**

Постройте модель на основе полносвязных слоёв для классификации Fashion MNIST из библиотеки torchvision (datasets).
Получите качество на тестовой выборке не ниже 88%

**Инструкция по выполнению задания**

Скачайте тренировочную и тестовою часть датасета
Постройте модель, выбрав стартовую архитектуру
Обучите модель и сверьте качество на тестовой части с заданным порогом
Изменяйте архитектуру модели пока качество на тестовой части не будет выше порога. Вариации архитектуры можно реализовать через изменение количества слоёв, количества нейронов в слоях и использование регуляризации. Можно использовать различные оптимизаторы.

In [1]:
import torchvision as tv
import torch
import time

In [2]:
f_mnist_train = tv.datasets.FashionMNIST('.', train=True, transform=tv.transforms.ToTensor(), download=True)
f_mnist_test  = tv.datasets.FashionMNIST('.', train=False, transform=tv.transforms.ToTensor(), download=True)

100%|██████████| 26421880/26421880 [00:03<00:00, 7054973.73it/s] 


Extracting ./FashionMNIST/raw/train-images-idx3-ubyte.gz to ./FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 116064.74it/s]


Extracting ./FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:02<00:00, 2196885.42it/s]


Extracting ./FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 16961725.84it/s]

Extracting ./FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./FashionMNIST/raw



In [3]:
f_mnist_train[0][0]

tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0039, 0.0000, 0.0000, 0.0510,
          0.2863, 0.0000, 0.0000, 0.0039, 0.0157, 0.0000,

In [4]:
train, test = torch.utils.data.DataLoader(f_mnist_train, 6000, shuffle=True), torch.utils.data.DataLoader(f_mnist_test, 1000, shuffle=True)

In [5]:
for X, y in train:
    print(X.shape, y.shape)
    break

for X, y in test:
    print(X.shape, y.shape)
    break

torch.Size([6000, 1, 28, 28]) torch.Size([6000])
torch.Size([1000, 1, 28, 28]) torch.Size([1000])


In [6]:
LR = 0.01

model = torch.nn.Sequential(
    torch.nn.Flatten(),
    torch.nn.Linear(784, 512),
    torch.nn.ReLU(),
    torch.nn.Linear(512, 256),
    torch.nn.ReLU(),
    torch.nn.Linear(256, 128),
    torch.nn.ReLU(),
    torch.nn.Linear(128, 10)
)


loss = torch.nn.CrossEntropyLoss(reduction='sum')
trainer = torch.optim.Adam(model.parameters(), LR)

In [7]:
NUM_EPOCHS = 20

for epoch in range(1, NUM_EPOCHS+1):

    start=time.time()
    train_loss, train_acc, train_n = 0., 0., 0
    test_loss, test_acc, test_n = 0., 0., 0
    

    model.train()
    for X, y in train:
        trainer.zero_grad()
        y_pred = model(X)
        l = loss(y_pred, y)
        l.backward()
        trainer.step()
        train_loss += l.item()
        train_acc += (y_pred.argmax(dim=1) == y).sum().item()
        train_n += len(X)
    
    model.eval()
    for X, y in test:
        y_pred = model(X)
        l = loss(y_pred, y)
        test_loss += l.item()
        test_acc += (y_pred.argmax(dim=1) == y).sum().item()
        test_n += len(X)
        
    print("\nepoch: {}, taked: {:.2f}, train_loss: {:.3f}, train_acc: {:.2f}%, test_loss: {:.3f}, test_acc: {:.2f}%".format(
        epoch, time.time() - start, train_loss / train_n, train_acc / train_n *100 , test_loss / test_n, test_acc / test_n *100))


epoch: 1, taked: 14.09, train_loss: 1.874, train_acc: 39.52%, test_loss: 1.008, test_acc: 59.35%

epoch: 2, taked: 14.18, train_loss: 0.872, train_acc: 65.95%, test_loss: 0.743, test_acc: 71.48%

epoch: 3, taked: 14.26, train_loss: 0.641, train_acc: 76.17%, test_loss: 0.585, test_acc: 77.56%

epoch: 4, taked: 13.85, train_loss: 0.517, train_acc: 81.10%, test_loss: 0.494, test_acc: 82.05%

epoch: 5, taked: 17.01, train_loss: 0.447, train_acc: 83.66%, test_loss: 0.453, test_acc: 83.37%

epoch: 6, taked: 14.37, train_loss: 0.401, train_acc: 85.26%, test_loss: 0.416, test_acc: 84.58%

epoch: 7, taked: 13.77, train_loss: 0.378, train_acc: 85.93%, test_loss: 0.400, test_acc: 85.58%

epoch: 8, taked: 13.94, train_loss: 0.353, train_acc: 86.89%, test_loss: 0.390, test_acc: 85.53%

epoch: 9, taked: 14.13, train_loss: 0.335, train_acc: 87.59%, test_loss: 0.392, test_acc: 85.81%

epoch: 10, taked: 13.75, train_loss: 0.322, train_acc: 88.12%, test_loss: 0.357, test_acc: 87.13%

epoch: 11, taked: 